In [ ]:
!pip install -r requirements.txt


# # below fixes some bugs introduced by some recent Colab changes
# !mkdir -p /usr/share/vulkan/icd.d
# !wget -q https://raw.githubusercontent.com/haosulab/ManiSkill2/main/docker/nvidia_icd.json
# !wget -q https://raw.githubusercontent.com/haosulab/ManiSkill2/main/docker/10_nvidia.json
# !mv nvidia_icd.json /usr/share/vulkan/icd.d
# !mv 10_nvidia.json /usr/share/glvnd/egl_vendor.d/10_nvidia.json
# # dependencies
# #!pip install setuptools==65.5.0
# !apt-get install -y --no-install-recommends libvulkan-dev
# !pip install mani_skill2
# !pip install --upgrade --no-cache-dir gdown

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import mani_skill2.envs
from tqdm.notebook import tqdm
import time
from pybullet.humanoid import HumanoidEnv
from pybullet.arm import ArmEnv


# Pybullet

In [ ]:
env = ArmEnv()
env.connect()
env.load_object()

obs = env.reset()
target_position = [0, 0.8, 1]
target_orientation = [0, np.pi, 0]
action={}
action['position'] = target_position
action['orientation'] = target_orientation
action['gripper'] = [0.1, 1]
obs = env.step(action)

time.sleep(2)
env.get_robot_info()

In [ ]:
# env = HumanoidEnv(gui=True)
# obs = env.reset()
# for _ in range(1000):
#     action = env.action_space.sample()  # Random action
#     obs, reward, done, _ = env.step(action)
#     time.sleep(1./240.)
# env.close()

# Maniskill

In [ ]:
env_id = "PickCube-v0"
obs_mode = "rgbd"
control_mode = "pd_joint_delta_pos"
reward_mode = "dense"

env = gym.make(env_id,
               obs_mode=obs_mode,
               reward_mode=reward_mode,
               control_mode=control_mode,
               enable_shadow=False)
obs, _ = env.reset()
print("Action Space:", env.action_space)

# take a look at the current state
img = env.unwrapped.render_cameras()
plt.figure(figsize=(10,6))
plt.title("Current State viewed through all RGB and Depth cameras")
plt.imshow(img)

In [ ]:
from mani_skill2.utils.wrappers import RecordEpisode
env = RecordEpisode(
    env,
    "./videos", # the directory to save replay videos and trajectories to
    info_on_video=True # when True, will add informative text onto the replay video such as step counter, reward, and other metrics
)

# step through the environment with random actions
obs, _ = env.reset()
for i in tqdm(range(100)):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

env.flush_video() # Save the video
env.close()
from IPython.display import Video
Video("./videos/0.mp4", embed=True) # Watch our replay